# Hent alle tabeller for KPI, og vis default uttrekk

### Nytt i API2
1. muligheten for å hente et default datasett med en URL uten spørreparametre. Norsk og JSON-stat2 satt i [/config](https://data.ssb.no/api/pxwebapi/v2-beta/config)
2. /navigation gir mulighet til å angi kortnavn for en statstikk direkte uten å traversere treet. (Det er mulig /navigation blir tatt vekk i endelig versjon. Da vil dette eksempelet ikke fungere)

Eksempelet henter JSON-stat uten det dedikerte biblioteket pyjstat. Her får en variabelkodene, de tilhørende tekstene vises som _label.

Takk til [Carl Corneill](https://github.com/aecorn) for eksempel på funksjoner, som jeg har bearbeidet litt bl.a. ved hjelp av Claude.

In [1]:
import requests
import pandas as pd
import itertools # itertools is a part of Python

 KPI etter måned

Funksjon for å hente alle tabeller innen en statistikk

In [2]:
def get_table_ids(shortname: str) -> list[str]:
    response = requests.get(f"https://data.ssb.no/api/pxwebapi/v2-beta/navigation/{shortname}")
    shortname_content = response.json()
    folder_contents = shortname_content.get("folderContents", [])
    return [x for x in [table.get("id") for table in folder_contents] if x.isdigit() and len(x) == 5]


In [3]:
shortname = 'kpi'

In [4]:
get_table_ids(shortname)

['03013',
 '11117',
 '05327',
 '11119',
 '06444',
 '08981',
 '08183',
 '03014',
 '11118',
 '11120',
 '05335',
 '08184',
 '03700',
 '10493',
 '03702',
 '10494']

Funksjon for å hente en tabell fra API2 Beta.

In [5]:
def fetch_data_and_metadata(tableid: str) -> dict:
    response = requests.get(f'https://data.ssb.no/api/pxwebapi/v2-beta/tables/{tableid}/data')

    # the whole dataset as JSON-stat2
    js2_data = response.json()

    # Get the dimensions of the dataset
    dimensions = {
            dim_name: list(dim_info["category"]["index"].keys()) 
            for dim_name, dim_info in js2_data["dimension"].items()
        }
    # Get the code to the dataframe
    dimension_combinations = list(itertools.product(*list(dimensions.values())))
    df = pd.DataFrame(dimension_combinations, columns=dimensions.keys())
    
    # Get the textlabels and add them next to codecolumn
    dimensions_labels = {
            dim_name: dim_info["category"]["label"] 
            for dim_name, dim_info in js2_data["dimension"].items()
        }
    for i, (col, codelist) in enumerate(dimensions_labels.items()):
            label_colname = f"{col}_label"
            df.insert((i*2), label_colname, df[col].map(codelist))
    
    # Add values to the dataframe
    df["Value"] = js2_data["value"]

    # Get metadata from dataset 
    metadata = dict(
        table_id = js2_data['extension']['px']['tableid'],
        short_title = js2_data['extension']['px']['contents'],
        title = js2_data['label'],
        source = js2_data['source'],
        last_update = js2_data['updated'],
        # include footnote, needs post processing
        # note = js2_data['note']
        )
        
    return {
        'dataframe': df, 
        'metadata' : metadata
        }

Henter default datasett for den mest brukte KPI-tabellen, med ID 03013

In [6]:
fetch_data_and_metadata('03013')['dataframe']


,ContentsCode_label,ContentsCode,Tid_label,Tid,Konsumgrp_label,Konsumgrp,Value
0,Konsumprisindeks (2015=100),KpiIndMnd,2025M02,2025M02,Totalindeks,TOTAL,137.0
1,Månedsendring (prosent),Manedsendring,2025M02,2025M02,Totalindeks,TOTAL,1.4
2,12-måneders endring (prosent),Tolvmanedersendring,2025M02,2025M02,Totalindeks,TOTAL,3.6
3,Konsumprisindeks (vekter),KpiVektMnd,2025M02,2025M02,Totalindeks,TOTAL,1000.0


Henter JSON-stat2 metadata for tabellen

In [7]:
fetch_data_and_metadata('03013')['metadata']

{'table_id': '03013',
 'short_title': '03013: Konsumprisindeks,',
 'title': '03013: Konsumprisindeks, etter statistikkvariabel, måned og konsumgruppe',
 'source': 'Statistisk sentralbyrå',
 'last_update': '2025-03-10T07:00:00Z'}

### Hent alle tabeller med metadata og vis 2 første linjer av datasett.

In [8]:
for tableid in get_table_ids(shortname):
    print(f"\n\n meta:\n{fetch_data_and_metadata(tableid)['metadata']}")
    display(fetch_data_and_metadata(tableid)['dataframe'].head(2))
    



 meta:
{'table_id': '03013', 'short_title': '03013: Konsumprisindeks,', 'title': '03013: Konsumprisindeks, etter statistikkvariabel, måned og konsumgruppe', 'source': 'Statistisk sentralbyrå', 'last_update': '2025-03-10T07:00:00Z'}


,ContentsCode_label,ContentsCode,Tid_label,Tid,Konsumgrp_label,Konsumgrp,Value
0,Konsumprisindeks (2015=100),KpiIndMnd,2025M02,2025M02,Totalindeks,TOTAL,137.0
1,Månedsendring (prosent),Manedsendring,2025M02,2025M02,Totalindeks,TOTAL,1.4




 meta:
{'table_id': '11117', 'short_title': '11117: Konsumprisindeks for varer og tjenester,', 'title': '11117: Konsumprisindeks for varer og tjenester, etter leveringssektor, måned og statistikkvariabel', 'source': 'Statistisk sentralbyrå', 'last_update': '2025-03-10T07:00:00Z'}


,Leveringssektor_label,Leveringssektor,Tid_label,Tid,ContentsCode_label,ContentsCode,Value
0,Varer,B1,2025M02,2025M02,Konsumprisindeksen (2015=100),LevIndMnd,139.9
1,Varer,B1,2025M02,2025M02,Månedsendring (prosent),Manedsendring,2.1




 meta:
{'table_id': '05327', 'short_title': '05327: KPI-JA og KPI-JAE,', 'title': '05327: KPI-JA og KPI-JAE, etter konsumgruppe, måned og statistikkvariabel', 'source': 'Statistisk sentralbyrå', 'last_update': '2025-03-10T07:00:00Z'}


...

,KonsumgrpH_label,KonsumgrpH,ContentsCode_label,ContentsCode,Tid_label,Tid,Value
0,Matvarer og alkoholfrie drikkevarer,K01,Harmonisert konsumprisindeks med konstante avg...,HkpilndMnd,2024M02,2024M02,130.1
1,Matvarer og alkoholfrie drikkevarer,K01,Harmonisert konsumprisindeks med konstante avg...,HkpilndMnd,2024M03,2024M03,127.8




 meta:
{'table_id': '03702', 'short_title': '03702: Harmonisert konsumprisindeks (2015=100),', 'title': '03702: Harmonisert konsumprisindeks (2015=100), etter statistikkvariabel, år og konsumgruppe', 'source': 'Statistisk sentralbyrå', 'last_update': '2025-02-10T07:00:00Z'}


,ContentsCode_label,ContentsCode,Tid_label,Tid,Konsumgrp_label,Konsumgrp,Value
0,Harmonisert konsumprisindeks,HkpiIndAar,2024,2024,Totalindeks,K_TOTAL,135.4
1,Harmonisert konsumprisindeks (vekter),HkpiVektAar,2024,2024,Totalindeks,K_TOTAL,1000.0




 meta:
{'table_id': '10494', 'short_title': '10494: Harmonisert konsumprisindeks for Norge, med konstante avgifter (HKPI-CT) (2015=100),', 'title': '10494: Harmonisert konsumprisindeks for Norge, med konstante avgifter (HKPI-CT) (2015=100), etter konsumgruppe, statistikkvariabel og år', 'source': 'Statistisk sentralbyrå', 'last_update': '2025-01-10T07:00:00Z'}


,KonsumgrpH_label,KonsumgrpH,ContentsCode_label,ContentsCode,Tid_label,Tid,Value
0,Matvarer og alkoholfrie drikkevarer,K01,Harmonisert konsumprisindeks med konstante avg...,HkpilndMnd,2013,2013,94.4
1,Matvarer og alkoholfrie drikkevarer,K01,Harmonisert konsumprisindeks med konstante avg...,HkpilndMnd,2014,2014,97.3
